# Linear optimization with scipy and pulp

(This tutorial is heavely based on: https://realpython.com/linear-programming-python/ and https://benalexkeen.com/linear-programming-with-python-and-pulp-part-1/ . For a more detailed introductio to linear programming check the Book: Operation research Applications and Algorithms, Wiston)

Linear optimization techniques are related to _linear_ problems both in the objective function and the restrictions. Of course you can solve them using the more general scipy functions like `minimize`, except for particular cases like integer binary variables. Instead, for all these linear problems, it is much better to rely of libraries optimized for these situations. 

Linear problems can also require integer values for some (or all variables). Those problems are called **Mixed integer programming** . A very special case is when one (or more variables) must be boolean, with values 0 or 1 only. Those **binary** problems are also  common in linear programming and are used, for instance, to decide if something must be build or not. 

The basic technique used in linear programming is the simplex method. An equivalent one is the interior point method. Furthermore , there are variants for other problems with bounds and restrictions, like the brand-and-bound or branch-and-cut methods. See https://en.wikipedia.org/wiki/Linear_programming  

For linear problems with continuous variables, we will use scipy. But later, for integer or binary variables, we will rely on specialized external libs like [pulp](https://coin-or.github.io/pulp/). Please install it using 
```
conda install pulp
```
on the anaconda prompt, or use the anaconda navigator, or just run the next cell. There are other libraries that you can use (free or paid). Some examples are [pyomo](https://pyomo.readthedocs.io/en/stable/index.html) (this one can actually inter-operate with gams), and [CVXOPT](https://cvxopt.org/userguide/intro.html) 


In [2]:
!conda install -y pulp -c conda-forge

Solving environment: done

# All requested packages already installed.



## Introduction
We will solve the following problem using scipy linear programming facilities.
$$ 
\text{Maximize:}\  z = x+2y
$$
subject to
\begin{align}
2x +y &\le 20 \\
-4x + 5y &\le 10\\
-x + 2y &\ge -2\\
x &\ge 0\\
y &\ge 0
\end{align}
In this problem $z$ is the objective function and the variables $x, y$ are called the decision variables, which are subject to several constraints. 

The feasible solution region, that is, the region where the solution must be according to the constraints, is 
 <img src="fig/ex1-feasible-region.png" alt="Feasible region" width="500" height="600"> 

If you put an equality as restriction, then that restricts even more the feasible region and must pass through the area highlighted before.

Following https://realpython.com/linear-programming-python/, we will start by solving a typical allocation problem.

## Example : Resource allocation problem
Imagine that you have a factory that produces four kinds of products. The daily amount produced, for each product, are $x_1, x_2, x_3, x_4$. The goal is to optimize the profit by finding the optimal values for daily production, under the following conditions:
- The profit per unit are 20, 12, 40 and 25. 
- The total number of units per day cannot be larger than 50.
- There are two raw materials available, A and B, to produce the units. The raw material used per unit is as follows:
  - first unit : 3 units of A.
  - second unit: 2 of A and 1 of B
  - third unit : 1 of A and 2 of B
  - fourth unit: 3 of B
- Due to transportation and other constraints, raw material A cannot exceed 100 units, while raw material B cannot exceed 90 units. 

This way the problem can be stated as 
$$
\text{Maximize: } 20x_1 + 12x_2 + 40x_3 +25x_4
$$
subject to
\begin{align}
x_1 + x_2 + x_3 + x_4 &\le 50\\
3x_1 + 2x_2 + x_3 &\le 100\\
x_2 + 2x_3 +3x_4 &\le 90\\
x_1, x_2, x_3, x_4 \ge 0
\end{align}

### Solution with scipy
Let's solve it using scipy linprog:

In [4]:
import numpy as np
from scipy.optimize import linprog
obj = [-20, -12, -40, -25] # objetive function coefficients
# coefficients for inequalities: left hand side
lhs_ineq = [[1, 1, 1, 1],
           [3, 2, 1, 0],
           [0, 1, 2, 3]]
# coefficients for inequalities: right hand side
rhs_ineq = [50, 
            100, 
            90]  
# bounds
bnd = [(0, np.inf), # x1 bounds
       (0, np.inf), # x2 bounds
       (0, np.inf), # x3 bounds
       (0, np.inf)] # x4 bounds

# Optimize
opt = linprog(c = obj, A_ub = lhs_ineq, b_ub = rhs_ineq,
              bounds=bnd, method="revised simplex")
opt

     con: array([], dtype=float64)
     fun: -1900.0
 message: 'Optimization terminated successfully.'
     nit: 2
   slack: array([ 0., 40.,  0.])
  status: 0
 success: True
       x: array([ 5.,  0., 45.,  0.])

In [ ]:
# Exercise : Solve the initial example and check the optimal position in the feasible region

### Solution with pulp
Scipy is good for small problems but when your scale increases you might better use some other libraries that can be helpful in some particular situations like: 
- larger problems that require a more succinct syntax than matrices and vectors
- problems with integer or binary variables
- need to test several solvers

Here, pulp allows to define the problem very easily and can handle all the restrictions stated above. The solution is as follows

In [6]:
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

# Create the model
model = LpProblem(name="resource-allocation", sense=LpMaximize)
# Initialize the decision variables
# can be x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 5)}
x1 = LpVariable(name="x1", lowBound=0, cat="Continuous")
x2 = LpVariable(name="x2", lowBound=0, cat="Continuous")
x3 = LpVariable(name="x3", lowBound=0, cat="Continuous")
x4 = LpVariable(name="x4", lowBound=0, cat="Continuous")
# Add the constraints to the model
model += (x1 + x2 + x3 + x4 <= 50, "Capacity")
model += (3*x1 + 2*x2 + x3 <= 100, "Raw A")
model += (x2 + 2*x3 + 3*x4 <= 90, "Raw B")
# Add the objective function to the model
obj_func = 20*x1 + 12*x2 + 40*x3 + 25*x4 # can be lpSum(20*x1, 12*x2, 40*x3, 25*x4)
model += obj_func
# print the system 
print(model)

resource-allocation:
MAXIMIZE
20*x1 + 12*x2 + 40*x3 + 25*x4 + 0
SUBJECT TO
Capacity: x1 + x2 + x3 + x4 <= 50

Raw_A: 3 x1 + 2 x2 + x3 <= 100

Raw_B: x2 + 2 x3 + 3 x4 <= 90

VARIABLES
x1 Continuous
x2 Continuous
x3 Continuous
x4 Continuous



In [7]:
# Now solve the system
status = model.solve()
# Get the results
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

print(f"{x1.name} = {x1.value()}")
print(f"{x2.name} = {x2.value()}")
print(f"{x3.name} = {x3.value()}")
print(f"{x4.name} = {x4.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 1900.0
x1 = 5.0
x2 = 0.0
x3 = 45.0
x4 = 0.0
Capacity: 0.0
Raw_A: -40.0
Raw_B: 0.0


### Extra complication: Binary (decision) variables
Now let's assume a problem with the machinery, so now it is not possible to produce products $x_1$ and $x_3$ at the same time. That is, if $x_1$ is being produced, then $x_3$ must be zero, and viceversa. In this case we need to introduce two binary variables that will help us model this case:

In [10]:
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

# Create the model
model = LpProblem(name="resource_allocation_non_parallel_production", sense=LpMaximize)
# Initialize the decision variables
# can be x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 5)}
x1 = LpVariable(name="x1", lowBound=0, cat="Continuous")
x2 = LpVariable(name="x2", lowBound=0, cat="Continuous")
x3 = LpVariable(name="x3", lowBound=0, cat="Continuous")
x4 = LpVariable(name="x4", lowBound=0, cat="Continuous")

# Binary decision variables
y1 = LpVariable(name="y1", cat="Binary")
y3 = LpVariable(name="y3", cat="Binary")

# Add the constraints to the model
model += (x1 + x2 + x3 + x4 <= 50, "Capacity")
model += (3*x1 + 2*x2 + x3 <= 100, "Raw A")
model += (x2 + 2*x3 + 3*x4 <= 90, "Raw B")

# non parallel production
model += (y1 + y3 <= 1, "y constraint")
model += (x3 <= y3*50, "x3 constraint")
model += (x1 <= y1*50, "x1 constraint")

# Add the objective function to the model
obj_func = 20*x1 + 12*x2 + 40*x3 + 25*x4 # can be lpSum(20*x1, 12*x2, 40*x3, 25*x4)
model += obj_func
# print the system 
print(model)

resource_allocation_non_parallel_production:
MAXIMIZE
20*x1 + 12*x2 + 40*x3 + 25*x4 + 0
SUBJECT TO
Capacity: x1 + x2 + x3 + x4 <= 50

Raw_A: 3 x1 + 2 x2 + x3 <= 100

Raw_B: x2 + 2 x3 + 3 x4 <= 90

y_constraint: y1 + y3 <= 1

x3_constraint: x3 - 50 y3 <= 0

x1_constraint: x1 - 50 y1 <= 0

VARIABLES
x1 Continuous
x2 Continuous
x3 Continuous
x4 Continuous
0 <= y1 <= 1 Integer
0 <= y3 <= 1 Integer



In [11]:
# Now solve the system
status = model.solve()
# Get the results
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

print(f"{x1.name} = {x1.value()}")
print(f"{x2.name} = {x2.value()}")
print(f"{x3.name} = {x3.value()}")
print(f"{x4.name} = {x4.value()}")
print(f"{y1.name} = {y1.value()}")
print(f"{y3.name} = {y3.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

status: 1, Optimal
objective: 1800.0
x1 = 0.0
x2 = 0.0
x3 = 45.0
x4 = 0.0
y1 = 0
y3 = 1
Capacity: -5.0
Raw_A: -55.0
Raw_B: 0.0
y_constraint: 0
x3_constraint: -5.0
x1_constraint: 0.0


Check another example at http://benalexkeen.com/linear-programming-with-python-and-pulp-part-4/ .

## Exercises

### Resource allocation 
(From http://benalexkeen.com/linear-programming-with-python-and-pulp-part-3/ - Please do not check the solution before trying the exercise by yourself) . 

In a car factory they run for 30 days producing some amount of cars type A and B. To build them, they have a robot, 2 engineers and 1 detailer. The detailer works only 21 days a month. Each car 
needs different resources: 

| resource  | car A  | car B  | 
|:-:|:-:|:-:|
| Robot  | 3 days  | 4 days  |  
| Engineer  | 5 days  | 6 days  | 
|  Detailer | 1.5 days  | 3 days  | 

Car A profit is 30000, while car B is 45000. 

Maximize the profit using pulp. You must get A = 2, B = 6, and profit of 330000. 

In [16]:
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

# Create the model
model = LpProblem(name="resource-allocation", sense=LpMaximize)
# Initialize the decision variables
# can be x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 5)}
NA = LpVariable(name="NA", lowBound=0, cat="Integer")
NB = LpVariable(name="NB", lowBound=0, cat="Integer")
# Add the constraints to the model
model += (3*NA + 4*NB <= 30, "Robot")
model += (5*NA + 6*NB <= 60, "Engineers")
model += (1.5*NA + 3*NB <= 21, "Detailer")
# Add the objective function to the model
model += (30000*NA + 45000*NB, "profit")
# print the system 
print(model)
# Now solve the system
status = model.solve()
# Get the results
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

print(f"{NA.name} = {NA.value()}")
print(f"{NB.name} = {NB.value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")

resource-allocation:
MAXIMIZE
30000*NA + 45000*NB + 0
SUBJECT TO
Robot: 3 NA + 4 NB <= 30

Engineers: 5 NA + 6 NB <= 60

Detailer: 1.5 NA + 3 NB <= 21

VARIABLES
0 <= NA Integer
0 <= NB Integer

status: 1, Optimal
objective: 330000
NA = 2
NB = 6
Robot: 0
Engineers: -14
Detailer: 0.0


### Basic linear programming
<img src="fig/exercise-basic.png" alt="Exercise" width="500" height="600">

### Teacher distribution (Wiston, 7-28)
<img src="fig/exercise-teachers.png" alt="Exercise" width="500" height="600">
<img src="fig/exercise-teachers-table.png" alt="Exercise" width="500" height="600">

In [19]:
# %load teachers-solution.py
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable

# Create the model
model = LpProblem(name="teacher-allocation", sense=LpMaximize)
# Initialize the decision variables
# can be x = {i: LpVariable(name=f"x{i}", lowBound=0) for i in range(1, 5)}
x = [[LpVariable(name=f"x{i}{j}", lowBound=0, cat="Binary") for j in range(0,6)] for i in range(0,3)]
# Add the constraints to the model
model += (lpSum([x[0][0], x[0][1], x[0][2], x[0][3], x[0][4], x[0][5]]) == 2, "teacher A")
model += (lpSum([x[1][0], x[1][1], x[1][2], x[1][3], x[1][4], x[1][5]]) == 2, "teacher B")
model += (lpSum([x[2][0], x[2][1], x[2][2], x[2][3], x[2][4], x[2][5]]) == 2, "teacher C")
model += (lpSum([x[0][0], x[0][1], x[0][2], x[0][3], x[0][4], x[0][5],
                 x[1][0], x[1][1], x[1][2], x[1][3], x[1][4], x[1][5],
                 x[2][0], x[2][1], x[2][2], x[2][3], x[2][4], x[2][5]]) == 6, "total hours")
# Add the objective function to the model
obj_func = lpSum([8*x[0][0], 7*x[0][1], 6*x[0][2], 5*x[0][3], 6*x[0][4], 7*x[0][5],
                 9*x[1][0], 9*x[1][1], 8*x[1][2], 8*x[1][3], 4*x[1][4], 4*x[1][5],
                 7*x[2][0], 6*x[2][1], 9*x[2][2], 6*x[2][3], 9*x[2][4], 9*x[2][5]]) 
model += obj_func
# print the system 
print(model)
# Now solve the system
status = model.solve()
# Get the results
print(f"status: {model.status}, {LpStatus[model.status]}")
print(f"objective: {model.objective.value()}")

for ii in range(0, 3):
    for jj in range(0, 6):
        print(f"{x[ii][jj].name} = {x[ii][jj].value()}")

for name, constraint in model.constraints.items():
    print(f"{name}: {constraint.value()}")


teacher-allocation:
MAXIMIZE
8*x00 + 7*x01 + 6*x02 + 5*x03 + 6*x04 + 7*x05 + 9*x10 + 9*x11 + 8*x12 + 8*x13 + 4*x14 + 4*x15 + 7*x20 + 6*x21 + 9*x22 + 6*x23 + 9*x24 + 9*x25 + 0
SUBJECT TO
teacher_A: x00 + x01 + x02 + x03 + x04 + x05 = 2

teacher_B: x10 + x11 + x12 + x13 + x14 + x15 = 2

teacher_C: x20 + x21 + x22 + x23 + x24 + x25 = 2

total_hours: x00 + x01 + x02 + x03 + x04 + x05 + x10 + x11 + x12 + x13 + x14
 + x15 + x20 + x21 + x22 + x23 + x24 + x25 = 6

VARIABLES
0 <= x00 <= 1 Integer
0 <= x01 <= 1 Integer
0 <= x02 <= 1 Integer
0 <= x03 <= 1 Integer
0 <= x04 <= 1 Integer
0 <= x05 <= 1 Integer
0 <= x10 <= 1 Integer
0 <= x11 <= 1 Integer
0 <= x12 <= 1 Integer
0 <= x13 <= 1 Integer
0 <= x14 <= 1 Integer
0 <= x15 <= 1 Integer
0 <= x20 <= 1 Integer
0 <= x21 <= 1 Integer
0 <= x22 <= 1 Integer
0 <= x23 <= 1 Integer
0 <= x24 <= 1 Integer
0 <= x25 <= 1 Integer

status: 1, Optimal
objective: 51
x00 = 1
x01 = 0
x02 = 0
x03 = 0
x04 = 0
x05 = 1
x10 = 1
x11 = 1
x12 = 0
x13 = 0
x14 = 0
x15 = 0
x20